https://huggingface.co/docs/transformers/en/training

In [ ]:
!pip install -qq adapters datasets evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 37.2 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset("hrithikpiyush/acl-arc")
dataset = dataset.rename_column(original_column_name="intent", new_column_name="labels")

train_data = dataset['train']
val_data = dataset['validation']
test_data = dataset['test']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1688 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/114 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/139 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("allenai/cs_roberta_base")

def tokenize_function(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True)

tokenized_train_data = train_data.map(tokenize_function, batched=True, batch_size=len(train_data))
tokenized_val_data = val_data.map(tokenize_function, batched=True, batch_size=len(val_data))
tokenized_test_data = test_data.map(tokenize_function, batched=True, batch_size=len(test_data))

tokenizer_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Map:   0%|          | 0/1688 [00:00<?, ? examples/s]

Map:   0%|          | 0/114 [00:00<?, ? examples/s]

Map:   0%|          | 0/139 [00:00<?, ? examples/s]

In [ ]:
tokenized_train_data.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_val_data.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_test_data.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

In [ ]:
from transformers import AutoModelForSequenceClassification

num_labels = len(set(train_data['labels']))
model = AutoModelForSequenceClassification.from_pretrained("allenai/cs_roberta_base", num_labels=num_labels)

pytorch_model.bin:   0%|          | 0.00/656M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at allenai/cs_roberta_base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import numpy as np
import evaluate

# https://huggingface.co/spaces/evaluate-metric/f1/blob/main/f1.py
# Example 5-A multi-label example
metric = evaluate.load("f1", "multilabel")

In [ ]:
# https://medium.com/@rakeshrajpurohit/customized-evaluation-metrics-with-hugging-face-trainer-3ff00d936f99

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    precision_macro = precision_score(labels, preds, average='macro')
    precision_weighted = precision_score(labels, preds, average='weighted')
    recall_macro = recall_score(labels, preds, average='macro')
    recall_weighted = recall_score(labels, preds, average='weighted')
    f1_macro = f1_score(labels, preds, average='macro')
    f1_weighted = f1_score(labels, preds, average='weighted')

    return {
        'accuracy': accuracy,
        'precision_macro': precision_macro,
        'precision_weighted': precision_weighted,
        'recall_macro': recall_macro,
        'recall_weighted': recall_weighted,
        'f1_macro': f1_macro,
        'f1_weighted': f1_weighted
    }

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=5,  # <- change from 6 to 5
    per_device_train_batch_size=32,  # <- default is 8 idk what is better
    per_device_eval_batch_size=32,  # <- default is 8
    auto_find_batch_size=True,  # automates lowering the batch size to resolve out-of-memory errors
    logging_steps=100,
    output_dir="./training_output_3",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
    evaluation_strategy="epoch"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_val_data,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision Macro,Precision Weighted,Recall Macro,Recall Weighted,F1 Macro,F1 Weighted
1,1.162700,0.977320,0.675439,0.306085,0.602854,0.355529,0.675439,0.321779,0.628875
2,0.862700,0.721933,0.763158,0.607556,0.738041,0.571543,0.763158,0.585291,0.747887
3,0.516000,0.627701,0.824561,0.892742,0.830170,0.676002,0.824561,0.734646,0.814944
4,0.340700,0.699648,0.789474,0.731551,0.788798,0.664703,0.789474,0.687900,0.786708
5,0.169400,0.739469,0.824561,0.695156,0.831261,0.719956,0.824561,0.705789,0.827403


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning

TrainOutput(global_step=530, training_loss=0.5821954538237374, metrics={'train_runtime': 730.2111, 'train_samples_per_second': 11.558, 'train_steps_per_second': 0.726, 'total_flos': 2220737060782080.0, 'train_loss': 0.5821954538237374, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.739469051361084,
 'eval_accuracy': 0.8245614035087719,
 'eval_precision_macro': 0.6951559934318555,
 'eval_precision_weighted': 0.8312613430127042,
 'eval_recall_macro': 0.719955609362389,
 'eval_recall_weighted': 0.8245614035087719,
 'eval_f1_macro': 0.7057886783496539,
 'eval_f1_weighted': 0.8274025931278819,
 'eval_runtime': 3.2215,
 'eval_samples_per_second': 35.387,
 'eval_steps_per_second': 1.242,
 'epoch': 5.0}

In [ ]:
trainer.predict(tokenized_test_data)

PredictionOutput(predictions=array([[-4.51938957e-01, -9.86706018e-01,  6.13344717e+00,
        -1.19981623e+00, -4.53629017e-01, -1.01128054e+00],
       [ 6.28635550e+00, -1.14400637e+00, -1.24949849e+00,
        -1.49473894e+00, -9.53521132e-01, -1.70100617e+00],
       [-9.98479962e-01, -1.39628518e+00, -2.76775455e+00,
        -2.20687345e-01,  4.59520388e+00, -3.53021652e-01],
       [ 6.21135616e+00, -1.30626619e+00, -4.70813960e-01,
        -1.50517404e+00, -1.32629263e+00, -1.70758414e+00],
       [-1.07249416e-01, -1.44525969e+00, -1.53054833e+00,
        -7.95091510e-01, -1.30963016e+00,  5.23880863e+00],
       [ 6.09955263e+00, -1.07247460e+00, -1.90693593e+00,
        -1.30231571e+00, -6.12870038e-01, -1.52304411e+00],
       [ 5.67755508e+00, -2.20340684e-01, -1.66096246e+00,
        -1.59194064e+00, -8.49350870e-01, -1.36107326e+00],
       [ 6.10150146e+00, -7.91077971e-01, -2.09442282e+00,
        -9.22740817e-01, -1.45943749e+00, -1.18928814e+00],
       [ 6.07220936

In [ ]:
from google.colab import runtime
runtime.unassign()